In [ ]:
BRANCH = 'r1.0.0rc1'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[nlp]

  Cloning https://github.com/NVIDIA/NeMo.git (to revision r1.0.0rc1) to /tmp/pip-install-yvbggizt/nemo-toolkit
  Running command git clone -q https://github.com/NVIDIA/NeMo.git /tmp/pip-install-yvbggizt/nemo-toolkit
  Running command git checkout -b r1.0.0rc1 --track origin/r1.0.0rc1
  Switched to a new branch 'r1.0.0rc1'
  Branch 'r1.0.0rc1' set up to track remote branch 'r1.0.0rc1' from 'origin'.
     |████████████████████████████████| 14.5MB 243kB/s 
     |████████████████████████████████| 686kB 46.3MB/s 
     |████████████████████████████████| 776.8MB 21kB/s 
     |████████████████████████████████| 12.8MB 247kB/s 
     |████████████████████████████████| 7.6MB 53.5MB/s 
     |████████████████████████████████| 7.0MB 29.6MB/s 
     |████████████████████████████████| 102kB 12.8MB/s 
     |████████████████████████████████| 133kB 58.7MB/s 
     |████████████████████████████████| 2.1MB 46.6MB/s 
     |████████████████████████████████| 1.2MB 37.7MB/s 
     |████████████████████████████████

In [ ]:
from nemo.collections import nlp as nemo_nlp
from nemo.utils.exp_manager import exp_manager

import os
import wget
import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf

In [ ]:
TASK = 'IMDB'
DATA_DIR = os.path.join(os.getcwd(), 'DATA_DIR')
RE_DATA_DIR = os.path.join(DATA_DIR, 'RE')
WORK_DIR = os.path.join(os.getcwd(), 'WORK_DIR')
MODEL_CONFIG = 'text_classification_config.yaml'

In [ ]:
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(os.path.join(DATA_DIR, 'RE'), exist_ok=True)
os.makedirs(WORK_DIR, exist_ok=True)

In [ ]:
# download the dataset
wget.download('https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz',
              os.path.join(DATA_DIR, 'data_re.tar'))

'/content/DATA_DIR/data_re.tar'

In [ ]:
!tar -xvf  {DATA_DIR}/data_re.tar -C {RE_DATA_DIR}

Streaming output truncated to the last 5000 lines.
aclImdb/train/unsup/44983_0.txt
aclImdb/train/unsup/44982_0.txt
aclImdb/train/unsup/44981_0.txt
aclImdb/train/unsup/44980_0.txt
aclImdb/train/unsup/44979_0.txt
aclImdb/train/unsup/44978_0.txt
aclImdb/train/unsup/44977_0.txt
aclImdb/train/unsup/44976_0.txt
aclImdb/train/unsup/44975_0.txt
aclImdb/train/unsup/44974_0.txt
aclImdb/train/unsup/44973_0.txt
aclImdb/train/unsup/44972_0.txt
aclImdb/train/unsup/44971_0.txt
aclImdb/train/unsup/44970_0.txt
aclImdb/train/unsup/44969_0.txt
aclImdb/train/unsup/44968_0.txt
aclImdb/train/unsup/44967_0.txt
aclImdb/train/unsup/44966_0.txt
aclImdb/train/unsup/44965_0.txt
aclImdb/train/unsup/44964_0.txt
aclImdb/train/unsup/44963_0.txt
aclImdb/train/unsup/44962_0.txt
aclImdb/train/unsup/44961_0.txt
aclImdb/train/unsup/44960_0.txt
aclImdb/train/unsup/44959_0.txt
aclImdb/train/unsup/44958_0.txt
aclImdb/train/unsup/44957_0.txt
aclImdb/train/unsup/44956_0.txt
aclImdb/train/unsup/44955_0.txt
aclImdb/train/unsup/4

In [ ]:
!rm '/content/DATA_DIR/data_re.tar'

In [ ]:
! ls -l $RE_DATA_DIR

total 4
drwxr-xr-x 4 7297 1000 4096 Jun 26  2011 aclImdb


In [ ]:
RE_DATA_DIR = '/content/DATA_DIR/RE/aclImdb'

In [ ]:
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/text_classification/data/import_datasets.py')

'import_datasets.py'

In [ ]:
! python import_datasets.py --dataset_name=imdb --source_data_dir={RE_DATA_DIR} --target_data_dir={RE_DATA_DIR}

[NeMo I 2021-04-17 09:07:21 import_datasets:50] Processing IMDB dataset and store at /content/DATA_DIR/RE/aclImdb


In [ ]:
# let's take a look at the training data
! head -n 5 {RE_DATA_DIR}/train.tsv

I watched Grendel the other night and am compelled to put together a Public Service Announcement.Grendel is another version of Beowulf, the thousand-year-old Anglo-Saxon epic poem. The SciFi channel has a growing catalog of inoffensive and uninteresting movies, and the previews promised an inauthentic low-budget mini-epic, but this one refused to let me switch channels. It was staggeringly, overwhelmingly, bad. I watched in fascination and horror at the train wreck you couldn't tear your eyes away from. I reached for a notepad and managed to capture part of what I was seeing. The following may contain spoilers or might just save your sanity. You've been warned.- Just to get it over with, Beowulf's warriors wore horned helmets. Trivial issue compared to what came after. It also appears that the helmets were in a bin and handed to whichever actor wandered by next. Fit, appearance and function were apparently irrelevant.- Marina Sirtis had obviously been blackmailed into doing the movie b

In [ ]:
# let's check the label mapping
! cat {RE_DATA_DIR}/label_ids.tsv

negative
positive


In [ ]:
# download the model's configuration file
config_dir = WORK_DIR + '/configs/'
os.makedirs(config_dir, exist_ok=True)
if not os.path.exists(config_dir + MODEL_CONFIG):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/text_classification/conf/' + MODEL_CONFIG, config_dir)
else:
    print ('config file is already exists')

In [ ]:
# this line will print the entire config of the model
config_path = f'{WORK_DIR}/configs/{MODEL_CONFIG}'
print(config_path)
config = OmegaConf.load(config_path)

/content/WORK_DIR/configs/text_classification_config.yaml


In [ ]:
config.model.train_ds.file_path = os.path.join(RE_DATA_DIR, 'train.tsv')
config.model.validation_ds.file_path = os.path.join(RE_DATA_DIR, 'test.tsv')
config.model.test_ds.file_path = os.path.join(RE_DATA_DIR, 'test.tsv')
config.trainer.max_epochs = 1
# Note: these are small batch-sizes - increase as appropriate to available GPU capacity
config.model.train_ds.batch_size=8
config.model.validation_ds.batch_size=8
config.model.dataset.num_classes=2

In [ ]:
print(OmegaConf.to_yaml(config))

trainer:
  gpus: 1
  num_nodes: 1
  max_epochs: 1
  max_steps: null
  accumulate_grad_batches: 1
  gradient_clip_val: 0.0
  amp_level: O0
  precision: 32
  accelerator: ddp
  log_every_n_steps: 1
  val_check_interval: 1.0
  resume_from_checkpoint: null
  num_sanity_val_steps: 0
  checkpoint_callback: false
  logger: false
model:
  nemo_path: text_classification_model.nemo
  tokenizer:
    tokenizer_name: ${model.language_model.pretrained_model_name}
    vocab_file: null
    tokenizer_model: null
    special_tokens: null
  language_model:
    pretrained_model_name: bert-base-uncased
    lm_checkpoint: null
    config_file: null
    config: null
  classifier_head:
    num_output_layers: 2
    fc_dropout: 0.1
  class_labels:
    class_labels_file: null
  dataset:
    num_classes: 2
    do_lower_case: false
    max_seq_length: 256
    class_balancing: null
    use_cache: false
  train_ds:
    file_path: /content/DATA_DIR/RE/aclImdb/train.tsv
    batch_size: 8
    shuffle: true
    num_samp

In [ ]:
config.model.task_name = TASK
config.model.output_dir = WORK_DIR
config.model.dataset.data_dir = RE_DATA_DIR

In [ ]:
print("Trainer config - \n")
print(OmegaConf.to_yaml(config.trainer))

Trainer config - 

gpus: 1
num_nodes: 1
max_epochs: 1
max_steps: null
accumulate_grad_batches: 1
gradient_clip_val: 0.0
amp_level: O0
precision: 32
accelerator: ddp
log_every_n_steps: 1
val_check_interval: 1.0
resume_from_checkpoint: null
num_sanity_val_steps: 0
checkpoint_callback: false
logger: false



In [ ]:
# lets modify some trainer configs
# checks if we have GPU available and uses it
cuda = 1 if torch.cuda.is_available() else 0
config.trainer.gpus = cuda

# for PyTorch Native AMP set precision=16
config.trainer.precision = 16 if torch.cuda.is_available() else 32

# remove distributed training flags
config.trainer.accelerator = 'DDP'

trainer = pl.Trainer(**config.trainer)

[NeMo W 2021-04-17 09:07:23 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
      warnings.warn(*args, **kwargs)
    
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


In [ ]:
config.exp_manager.exp_dir = WORK_DIR
exp_dir = exp_manager(trainer, config.get("exp_manager", None))

# the exp_dir provides a path to the current experiment for easy access
exp_dir = str(exp_dir)
exp_dir

[NeMo I 2021-04-17 09:07:24 exp_manager:208] Experiments will be logged at /content/WORK_DIR/TextClassification/2021-04-17_09-07-24
[NeMo I 2021-04-17 09:07:24 exp_manager:548] TensorboardLogger has been set up


'/content/WORK_DIR/TextClassification/2021-04-17_09-07-24'

In [ ]:
# complete list of supported BERT-like models
print(nemo_nlp.modules.get_pretrained_lm_models_list())

# specify BERT-like model, you want to use, for example, "megatron-bert-345m-uncased" or 'bert-base-uncased'
PRETRAINED_BERT_MODEL = "biomegatron-bert-345m-uncased"

['megatron-bert-345m-uncased', 'megatron-bert-345m-cased', 'megatron-bert-uncased', 'megatron-bert-cased', 'biomegatron-bert-345m-uncased', 'biomegatron-bert-345m-cased', 'bert-base-uncased', 'bert-large-uncased', 'bert-base-cased', 'bert-large-cased', 'bert-base-multilingual-uncased', 'bert-base-multilingual-cased', 'bert-base-chinese', 'bert-base-german-cased', 'bert-large-uncased-whole-word-masking', 'bert-large-cased-whole-word-masking', 'bert-large-uncased-whole-word-masking-finetuned-squad', 'bert-large-cased-whole-word-masking-finetuned-squad', 'bert-base-cased-finetuned-mrpc', 'bert-base-german-dbmdz-cased', 'bert-base-german-dbmdz-uncased', 'cl-tohoku/bert-base-japanese', 'cl-tohoku/bert-base-japanese-whole-word-masking', 'cl-tohoku/bert-base-japanese-char', 'cl-tohoku/bert-base-japanese-char-whole-word-masking', 'TurkuNLP/bert-base-finnish-cased-v1', 'TurkuNLP/bert-base-finnish-uncased-v1', 'wietsedv/bert-base-dutch-cased', 'distilbert-base-uncased', 'distilbert-base-uncased-

In [ ]:
# add the specified above model parameters to the config
config.model.language_model.pretrained_model_name = PRETRAINED_BERT_MODEL

In [ ]:
model = nemo_nlp.models.TextClassificationModel(cfg=config.model, trainer=trainer)

[NeMo I 2021-04-17 09:07:43 megatron_utils:269] Downloading from https://api.ngc.nvidia.com/v2/models/nvidia/biomegatron345muncased/versions/0/files/vocab.txt


Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.



[NeMo I 2021-04-17 09:07:46 text_classification_dataset:120] Read 25000 examples from /content/DATA_DIR/RE/aclImdb/train.tsv.
[NeMo I 2021-04-17 09:07:47 text_classification_dataset:233] *** Example ***
[NeMo I 2021-04-17 09:07:47 text_classification_dataset:234] example 0: ['I', 'was', 'treated', 'to', 'a', 'viewing', 'of', 'Cracker', 'Bag', 'last', 'night', 'before', 'a', 'preview', 'screening', 'of', "Disney's", 'Holes.', 'I', "don't", 'know', 'who', 'decided', 'to', 'show', 'it', 'but', "I'm", 'so', 'very', 'glad', 'they', 'did.', 'Cracker', 'Bag', 'is', 'an', 'absolute', 'gem,', 'a', 'snapshot', 'of', 'Australia', 'in', 'the', 'early', '80s', 'as', 'seen', 'through', 'a', "child's", 'eye.', 'The', '"conversations"', 'between', 'Eddie', 'and', 'her', 'brother', 'were', 'hilarious', 'and,', 'as', 'with', 'the', 'rest', 'of', 'the', 'film,', 'so', 'true', 'to', 'life.', 'Each', 'shot', 'brought', 'a', 'great', 'sense', 'of', 'nostalgia', 'as', 'it', 'reminded', 'me', 'of', 'my', 'ow

[NeMo W 2021-04-17 09:14:09 text_classification_dataset:246] Found 10286 out of 25000 sentences with more than 256 subtokens. Truncated long sentences from the end.


[NeMo I 2021-04-17 09:14:09 data_preprocessing:297] Some stats of the lengths of the sequences:
[NeMo I 2021-04-17 09:14:09 data_preprocessing:303] Min: 13 |                  Max: 257 |                  Mean: 203.72836 |                  Median: 222.0
[NeMo I 2021-04-17 09:14:09 data_preprocessing:305] 75 percentile: 257.00
[NeMo I 2021-04-17 09:14:09 data_preprocessing:306] 99 percentile: 257.00
[NeMo I 2021-04-17 09:14:10 text_classification_dataset:120] Read 25000 examples from /content/DATA_DIR/RE/aclImdb/test.tsv.
[NeMo I 2021-04-17 09:14:11 text_classification_dataset:233] *** Example ***
[NeMo I 2021-04-17 09:14:11 text_classification_dataset:234] example 0: ['Unforgiven', 'is', 'Clint', 'Eastwoods', 'last', 'tribute', 'to', 'the', 'once', 'great', 'west.', 'But', 'whilst', 'i', 'thought', 'this', 'was', 'going', 'to', 'be', 'good', 'and', 'raise', 'the', 'bar', 'for', 'future', 'westerns', 'to', 'come', 'i', 'was', 'sadly', 'mistaken.', 'Unforgiven,', 'though', 'simple', 'in', 

...

[NeMo I 2021-04-17 09:20:25 data_preprocessing:297] Some stats of the lengths of the sequences:
[NeMo I 2021-04-17 09:20:25 data_preprocessing:303] Min: 10 |                  Max: 257 |                  Mean: 202.23424 |                  Median: 219.0
[NeMo I 2021-04-17 09:20:25 data_preprocessing:305] 75 percentile: 257.00
[NeMo I 2021-04-17 09:20:25 data_preprocessing:306] 99 percentile: 257.00
[NeMo I 2021-04-17 09:20:26 text_classification_dataset:120] Read 25000 examples from /content/DATA_DIR/RE/aclImdb/test.tsv.
[NeMo I 2021-04-17 09:20:26 text_classification_dataset:233] *** Example ***
[NeMo I 2021-04-17 09:20:26 text_classification_dataset:234] example 0: ['Unforgiven', 'is', 'Clint', 'Eastwoods', 'last', 'tribute', 'to', 'the', 'once', 'great', 'west.', 'But', 'whilst', 'i', 'thought', 'this', 'was', 'going', 'to', 'be', 'good', 'and', 'raise', 'the', 'bar', 'for', 'future', 'westerns', 'to', 'come', 'i', 'was', 'sadly', 'mistaken.', 'Unforgiven,', 'though', 'simple', 'in', 

[NeMo W 2021-04-17 09:26:41 text_classification_dataset:246] Found 10031 out of 25000 sentences with more than 256 subtokens. Truncated long sentences from the end.


[NeMo I 2021-04-17 09:26:41 data_preprocessing:297] Some stats of the lengths of the sequences:
[NeMo I 2021-04-17 09:26:41 data_preprocessing:303] Min: 10 |                  Max: 257 |                  Mean: 202.23424 |                  Median: 219.0
[NeMo I 2021-04-17 09:26:41 data_preprocessing:305] 75 percentile: 257.00
[NeMo I 2021-04-17 09:26:41 data_preprocessing:306] 99 percentile: 257.00
[NeMo I 2021-04-17 09:26:42 megatron_utils:269] Downloading from https://api.ngc.nvidia.com/v2/models/nvidia/biomegatron345muncased/versions/0/files/MegatronBERT.pt
using world size: 1 and model-parallel size: 1 
using torch.float32 for parameters ...
-------------------- arguments --------------------
  adlr_autoresume ................. False
  adlr_autoresume_interval ........ 1000
  apply_query_key_layer_scaling ... False
  apply_residual_connection_post_layernorm  False
  attention_dropout ............... 0.1
  attention_softmax_in_fp32 ....... False
  batch_size ...................... Non

[NeMo W 2021-04-17 09:27:00 megatron_bert:153] Megatron-lm checkpoint version not found. Setting checkpoint_version to 0.


[NeMo I 2021-04-17 09:27:00 megatron_bert:160] Checkpoint loaded from from /root/.cache/torch/megatron/biomegatron-bert-345m-uncased


In [ ]:
print(config.model.nemo_path)

text_classification_model.nemo


In [ ]:
model.save_to(config.model.nemo_path)

In [ ]:
!cp '/content/text_classification_model.nemo' '/content/WORK_DIR/TextClassification/2021-04-17_09-07-24'

In [ ]:
# !rm '/content/text_classification_model.nemo'

In [ ]:
try:
    from google import colab
    COLAB_ENV = True
except (ImportError, ModuleNotFoundError):
    COLAB_ENV = False

# Load the TensorBoard notebook extension
if COLAB_ENV:
    %load_ext tensorboard
    %tensorboard --logdir {exp_dir}
else:
    print("To use tensorboard, please use this notebook in a Google Colab environment.")

In [ ]:
# import torch
# torch.cuda.empty_cache()

In [ ]:
# start model training
trainer.fit(model)

initializing ddp: GLOBAL_RANK: 0, MEMBER: 1/1


torch distributed is already initialized, skipping initialization ...
> initializing model parallel with size 1
> setting random seeds to 1234 ...
> initializing model parallel cuda seeds on global rank 0, model parallel rank 0, and data parallel rank 0 with model parallel seed: 3952 and data parallel seed: 1234
[NeMo I 2021-04-17 09:51:33 modelPT:685] Optimizer config = Adam (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.999]
        eps: 1e-08
        lr: 2e-05
        weight_decay: 0.01
    )
[NeMo I 2021-04-17 09:51:33 lr_scheduler:621] Scheduler "<nemo.core.optim.lr_scheduler.WarmupAnnealing object at 0x7f79d5957210>" 
    will be used during training (effective maximum steps = 3125) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: 0.1
    last_epoch: -1
    max_steps: 3125
    )



  | Name                  | Type                 | Params
---------------------------------------------------------------
0 | loss                  | CrossEntropyLoss     | 0     
1 | bert_model            | MegatronBertEncoder  | 334 M 
2 | classifier            | SequenceClassifier   | 1.1 M 
3 | classification_report | ClassificationReport | 0     
---------------------------------------------------------------
335 M     Trainable params
0         Non-trainable params
335 M     Total params


[NeMo I 2021-04-17 10:41:38 text_classification_model:166] val_report: 
    label                                                precision    recall       f1           support   
    label_id: 0                                             92.27      89.35      90.79      12500
    label_id: 1                                             89.68      92.51      91.07      12500
    -------------------
    micro avg                                               90.93      90.93      90.93      25000
    macro avg                                               90.97      90.93      90.93      25000
    weighted avg                                            90.97      90.93      90.93      25000
    


Epoch 0, global step 3124: val_loss reached 0.21875 (best 0.21875), saving model to "/content/WORK_DIR/TextClassification/2021-04-17_09-07-24/checkpoints/TextClassification---val_loss=0.22-epoch=0.ckpt" as top 3
Saving latest checkpoint...


1

In [ ]:
model.setup_test_data(test_data_config=config.model.test_ds)

[NeMo I 2021-04-17 10:46:25 text_classification_dataset:120] Read 25000 examples from /content/DATA_DIR/RE/aclImdb/test.tsv.
[NeMo I 2021-04-17 10:46:25 text_classification_dataset:233] *** Example ***
[NeMo I 2021-04-17 10:46:25 text_classification_dataset:234] example 0: ['Unforgiven', 'is', 'Clint', 'Eastwoods', 'last', 'tribute', 'to', 'the', 'once', 'great', 'west.', 'But', 'whilst', 'i', 'thought', 'this', 'was', 'going', 'to', 'be', 'good', 'and', 'raise', 'the', 'bar', 'for', 'future', 'westerns', 'to', 'come', 'i', 'was', 'sadly', 'mistaken.', 'Unforgiven,', 'though', 'simple', 'in', 'plot', 'it', 'falls', 'flat', 'on', 'characters', 'and', 'emotions', 'and', 'i', 'would', 'certainly', 'say', 'that', 'Unforgiven', 'is', 'Eastwoods', 'worst', 'film', 'to', 'date.', 'Eastwoods', 'himself', 'seems', 'too', 'old', 'to', 'play', 'the', 'part', 'of', 'acting', 'and', 'directing', 'which', 'also', 'adds', 'a', 'downfall', 'to', 'the', 'overall', 'look', 'of', 'the', 'film.All', 'the'

[NeMo W 2021-04-17 10:52:39 text_classification_dataset:246] Found 10031 out of 25000 sentences with more than 256 subtokens. Truncated long sentences from the end.


[NeMo I 2021-04-17 10:52:39 data_preprocessing:297] Some stats of the lengths of the sequences:
[NeMo I 2021-04-17 10:52:39 data_preprocessing:303] Min: 10 |                  Max: 257 |                  Mean: 202.23424 |                  Median: 219.0
[NeMo I 2021-04-17 10:52:39 data_preprocessing:305] 75 percentile: 257.00
[NeMo I 2021-04-17 10:52:39 data_preprocessing:306] 99 percentile: 257.00


In [ ]:
trainer.test(model)

[NeMo I 2021-04-17 11:01:36 text_classification_model:166] test_report: 
    label                                                precision    recall       f1           support   
    label_id: 0                                             92.28      89.35      90.79      12500
    label_id: 1                                             89.68      92.52      91.08      12500
    -------------------
    micro avg                                               90.94      90.94      90.94      25000
    macro avg                                               90.98      90.94      90.93      25000
    weighted avg                                            90.98      90.94      90.93      25000
    

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_f1': tensor(90.9360, device='cuda:0'),
 'test_loss': tensor(0.2187, device='cuda:0'),
 'test_precision': tensor(90.9360, device='cuda:0'),
 'test_recall': tensor(90.9360, device='cu

[{'test_f1': 90.93599700927734,
  'test_loss': 0.21865606307983398,
  'test_precision': 90.93599700927734,
  'test_recall': 90.93599700927734}]